# utils

> util functions.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, pwd, sys, json, yaml, atexit, tempfile, inspect

import numpy as np, pandas as pd
import anndata as ad

from rich.repr import auto as rich_auto
from dataclasses import dataclass, field, asdict
from typing import Optional, Union, Any, List, TypeAlias, Literal, Tuple

In [ ]:
#| export
def get_user() -> str:
    user = pwd.getpwuid(os.getuid())[0]
    return user

def collapse_user(path: str) -> str:
    prefix, rest = path.split(get_user())    
    return '~' + rest

def make_temp_file(**kwargs) -> tempfile.NamedTemporaryFile:
    temp = tempfile.NamedTemporaryFile(**kwargs)
    @atexit.register
    def delete_temp() -> None:
        temp.close()
    return temp

In [ ]:
#| export
from featherplot.types import (
    QUADFEATHER_REQUIRED_COLUMNS,
    QUADFEATHER_EXPECTED_COLUMNS,
)

In [ ]:
#| export
@rich_auto
@dataclass
class QuadFeatherRenamer:
    df: Optional[pd.DataFrame]
    
    # whether or not to actually do renaming
    # if False, this is equivalent to dry run
    do: Optional[bool] = True
    
    # whether or not to do just required or all expected columns
    use_all: Optional[bool] = True

    # NOTE: strictly required
    required_columns: Optional[List[str]] = field(default_factory=lambda : ['x', 'y'])
    
    # NOTE: assumed to be present
    expected_columns: Optional[List[str]] = field(default_factory=lambda : ['x', 'y', 'z'])

    def __rich_repr__(self):
        yield f'df.shape({self.df.shape})'

    def rename(
        self, 
        df:Optional[pd.DataFrame]=None, do:Optional[bool]=None
    ) -> Tuple[pd.DataFrame, dict]:
        
        if df is None:
            df = self.df.copy()

        if do is None:
            do = self.do

        needed = self.expected_columns if self.use_all else self.required_columns
        missing = sorted(list(set(needed) - set(df.columns)))
        
        renamed = dict()

        for i, col_name in enumerate(df.columns):
            if col_name not in self.expected_columns and len(missing) > 0:
                new_name = missing.pop(0)
                renamed[col_name] = dict(new=new_name, old=col_name, i=i)
        
        name_map = {k: v['new'] for k, v in renamed.items()}
        if not self.do:
            return df, name_map
        
        df = df.rename(name_map, axis=1)
        return df, name_map

## Column to Channel

In [ ]:
#| export
from featherplot.types import (
    Transform, Range, StringRange, Domain, SingleArgumentConditonal, TwoArgumentConditional,
    Conditional, ConditionalChannel, LambdaChannel, ConstantBool, ConstantNumber, ConstantColor,
    BooleanChannel, ConstantChannel, BasicChannel, BasicColorChannel, FunctionalChannel,
    CategoricalColorChannel, ColorChannel, RootChannel, BasicBooleanChannel
)
from typing import Callable, get_args, Dict

In [ ]:
#| export
from featherplot.types import (
    TypeGuardError, MissingKwargsError, LambdaChannelError,
    ConditionalTypeGuard, TransformTypeGuard, QuadFeatherColumnTypeGuard,
    
    ChannelCreationError, LambdaChannelError, ConditionalChannelError,
    ConstantChannelError, BasicChannelError, CategoricalChannelError, 
    CategoricalChannel
)

In [ ]:
#| export

def create_lambda_channel(field: str, col:pd.Series, **kwargs) -> LambdaChannel:
    lfunc = kwargs.get('lfunc', None)
    if lfunc is None:
        raise MissingKwargsError('Must provide lfunc (lambda function))')
    try:
        # attempt to create LambdaChannel
        domain = (col.min().tolist(), col.max().tolist())
        mapped = col.map(lfunc)
        range = (mapped.min().tolist(), mapped.max().tolist())
        channel = LambdaChannel(field=field, lfunc=lfunc, range=range, domain=domain, **kwargs)
    except:
        raise LambdaChannelError('Could not create LambdaChannel')
    return channel

def create_conditional_channel(field: str, col:pd.Series, **kwargs) -> ConditionalChannel:
     # NOTE: for ConditionalChannel
    a = kwargs.get('a', None)
    b = kwargs.get('b', None)
    op = kwargs.get('op', None)

    if a is None and op is None:
        raise MissingKwargsError('Must provide at least a and op (b optional)')

    is_a_number = pd.api.types.is_number(a)

    is_b_number_or_none = pd.api.types.is_number(b) or b is None

    is_op_valid = ConditionalTypeGuard.validate(op)

    if not all([is_a_number, is_b_number_or_none, is_op_valid]):
        raise ValueError('a must be a number, b must be a number or None, op must pass ConditionalTypeGuard')
    
    try:
        channel = ConditionalChannel(field=field, a=a, b=b, op=op, **kwargs)
    except:
        raise ConditionalChannelError('Could not create ConditionalChannel')

    return channel

def create_functional_channel(
    field: str, col:pd.Series, **kwargs
) -> FunctionalChannel:
    lce, cce = [None, None]
    try:
        channel = create_lambda_channel(field, col, **kwargs)
    except LambdaChannelError as lce:
        lce = lce
        pass

    try:
        channel = create_conditional_channel(field, col, **kwargs)
    except ConditionalChannelError as cce:
        cce = cce
        pass
    
    if lce is not None and cce is not None:
        if lce:
            raise lce
        if cce:
            raise cce
    return channel

def create_boolean_channel(field:str, col:pd.Series, **kwargs) -> BooleanChannel:
    bce, lce, cce = [None, None, None] 

    # NOTE: try to create Boolean channel
    try:
        if col.dtype == 'bool':
            # ConstantBool(constant='boolean')
            channel = BasicBooleanChannel(field=field, constant='boolean', **kwargs)
            return channel
    except:
        bce = ConstantChannelError('Could not create ConstantBool')

    if bce is not None:
        # NOTE: try to create Fucntional Channel instead
        try:
            channel = create_functional_channel(field, col, **kwargs)
        except LambdaChannelError as lce:
            lce = lce
            pass
        except ConditionalChannelError as cce:
            cce = cce
            pass

        if lce is not None or cce is not None:
            if lce:
                raise lce
            if cce:
                raise cce
        
        return channel
    return channel
        
def create_color_channel(field:str, col:pd.Series, **kwargs) -> ColorChannel:
    dtype = col.dtype.name
    # NOTE: force strings to be categories
    if dtype == 'object':
        col = col.astype('category')
        dtype = col.dtype.name

    try:
        if dtype == 'category':
            col = col.cat.as_ordered()
            domain = int(col.cat.codes.min().tolist()), int(col.cat.codes.max().tolist())

            cats = col.cat.categories.tolist() if len(col.cat.categories) <= 10 else None
            channel = CategoricalColorChannel(field=field, domain=domain, categories=cats, **kwargs)        
            return channel
        
    except:
        raise CategoricalChannelError('Could not create CategoricalColorChannel')
    
def create_root_channel(field:str, col:pd.Series, **kwargs) -> RootChannel:
    dtype = col.dtype.name
    channel = None
    match dtype:
        case 'bool':
            channel = create_boolean_channel(field, col, **kwargs)
        
        case 'category':
            # NOTE: this solve a lot of issues with
            # distinguishing between BasicColorChannel and CategoricalColorChannel
            # as BasicColorChannel has a *NUMERIC* domain and 
            # CategoricalColorChannel has strings as a domain
            # category works with both
            channel = create_color_channel(field, col, **kwargs)
        
        # NOTE: this forces strings to be categories
        case 'object':
            channel = create_color_channel(field, col, **kwargs)
        
        case _:
            # NOTE: not a bool, not a category, not a string, must be numeric
            # pd.api.types.is_number(a)
            channel = BasicChannel(field=field, domain=(col.min().tolist(), col.max().tolist()), **kwargs)            
    
    if channel is None:
        raise ChannelCreationError('Could not create RootChannel')
    return channel

@rich_auto
@dataclass
class SeriesToChannel:
    series: pd.Series
    is_sidecar: Optional[bool] = False
    alt_name: Optional[str] = None

    def convert(self):
        if self.alt_name is not None:
            return create_root_channel(self.series.name, self.series, human=self.alt_name)
        return create_root_channel(self.series.name, self.series)
    
    def is_category(self):
        channel = self.convert()
        return isinstance(channel, (CategoricalColorChannel, CategoricalChannel, BasicColorChannel))
    
    def as_category(self):
        self.series = self.series.astype('category')
        return self
    
    def to(self, to:str, **kwargs):
        alt_name = kwargs.get('alt_name', None)
        if alt_name is None and self.alt_name is not None:
            kwargs['alt_name'] = self.alt_name
         
        match to:
            case 'lambda':
                channel = create_lambda_channel(self.series.name, self.series, **kwargs)
            case 'conditional':
                channel = create_conditional_channel(self.series.name, self.series, **kwargs)
            case 'functional':
                channel = create_functional_channel(self.series.name, self.series, **kwargs)
            case 'boolean':
                channel = create_boolean_channel(self.series.name, self.series, **kwargs)
            case 'color':
                channel = create_color_channel(self.series.name, self.series, **kwargs)
            case 'root':
                channel = create_root_channel(self.series.name, self.series, **kwargs)
            case _:
                raise ValueError(f'Unknown channel type {to}')
        return channel
    
    def as_lambda(self, **kwargs):
        return self.to('lambda', **kwargs)
    
    def as_conditional(self, **kwargs):
        return self.to('conditional', **kwargs)
    
    def as_functional(self, **kwargs):
        return self.to('functional', **kwargs)
    
    def as_boolean(self, **kwargs):
        return self.to('boolean', **kwargs)
    
    def as_color(self, **kwargs):
        return self.to('color', **kwargs)
    
    def as_root(self, **kwargs):
        return self.to('root', **kwargs)
    
    def valid_types(self, **kwargs) -> Tuple[List[str], List[str]]:
        errors = []
        valids = []
        for t in ['lambda', 'conditional', 'functional', 'boolean', 'color', 'root']:
            try:
                self.to(t, **kwargs)
                valids.append(t)
            except:
                errors.append(t)
        return valids, errors

    def to_meta(self, **kwargs) -> dict:
        channel = self.convert()
        return channel.to_meta(**kwargs)

## DataFrame to MetaData

In [ ]:
#| export
@rich_auto
@dataclass
class DataFrameToMetadata:
    df: pd.DataFrame
    sidecars: Optional[List[str]] = field(default_factory=list)
    embedding: Optional[List[str]] = field(default_factory=lambda : ['x', 'y', 'z'])
    alt_names: Optional[Dict[str, str]] = field(default_factory=dict)
    
    include_index: Optional[bool] = True

    def __post_init__(self):
        if not self.sidecars:
            self.sidecars = self._default_sidecars()

        if not self.embedding:
            self.embedding = ['x', 'y', 'z']
        else:
            if not all([e in self.embedding for e in ['x', 'y', 'z']]):
                raise ValueError('Must have x, y, z in embedding')
            
    def _default_sidecars(self) -> List[str]:
        return sorted(list(set(self.df.columns) - set(self.embedding)))
    
    def _do_one(self, name:str, col:pd.Series, is_index:bool=False) -> Tuple[Union[Any, None], Union[Dict[str, Any], None]]:
        success = None
        failure = None
        
        is_sidecar = name in self.sidecars and not name in self.embedding
        alt_name = self.alt_names.get(name, None)
        
        try:
            s2c = SeriesToChannel(col, is_sidecar, alt_name)
            channel = s2c.convert()
            
            # NOTE: if channel COULD be categorical force it to be a category
            if (not is_index and self.df.index.name != name) and s2c.is_category():
                channel = s2c.as_category().convert()
                self.df.loc[:, name] = s2c.series.values


            success = channel
        except Exception as e:
            failure = dict(name=name, is_sidecar=is_sidecar, alt_name=alt_name, error=e)

        return success, failure

    def convert(self) -> Tuple[dict, List[Dict[str, Dict[str, Any]]]]:
        failed = []
        channels = {}

        if self.include_index:
            name = 'index' if self.df.index.name is None else self.df.index.name
            succ, fail = self._do_one(name, pd.Series(self.df.index), is_index=True)
            if succ is not None:
                channels[name] = succ
            else:
                failed.append(fail)

        for i, (name, col) in enumerate(self.df.iteritems()):            
            succ, fail = self._do_one(name, col)
            if succ is not None:
                if name not in channels:
                    channels[name] = succ                
            else:
                failed.append(fail)
                
        return channels, failed
    
    def to_dict(self) -> dict:
        channels, failed = self.convert()
        index = self.df.index.name if self.df.index.name is not None else 'index'
        n_points = self.df.shape[0]
        columns_metadata = dict()
        for k, v in channels.items():
            columns_metadata[k] = v.to_dict()

        d = dict(
            index=index,
            n_points=n_points,
            embedding=self.embedding,
            sidecars=self.sidecars,
            columns_metadata=columns_metadata,
            tiles_dir=None
        )
        return d
    
    def to_meta(self) -> dict:
        channels, failed = self.convert()
        index = self.df.index.name if self.df.index.name is not None else 'index'
        n_points = self.df.shape[0]
        columns_metadata = dict()
        for k, v in channels.items():
            columns_metadata[k] = v.to_meta()

        d = dict(
            index=index,
            n_points=n_points,
            embedding=self.embedding,
            sidecars=self.sidecars,
            columns_metadata=columns_metadata,
            tiles_dir=None
        )
        return d


## AnnData

### Mocking AnnData

In [ ]:
#| export
@rich_auto
@dataclass
class MockSingleCellData:
    label: Optional[str] = 'barcode'
    c_str: Optional[str] = 'condition'
    n_str: Optional[str] = 'norm'
    
    n_genes: Optional[int] = 100
    n_points: Optional[int] = 1000
    emb_name: Optional[str] = 'MOCK'
    emb_dims: Optional[int] = 3
    n_conditions: Optional[int] = 4
    hvg_ratio: Optional[int] = 5
    
    @property
    def emb_cols(self):
        arr = list(map(lambda i: f'{self.emb_name}_{i}', np.arange(self.emb_dims)))
        return np.array(arr)
    
    @property
    def gene_symbols(self):
        try:
            return self._gene_symbols
        except AttributeError:
            _str = 'gene_symbol'
            name = f'{_str}s'
            func = lambda i: f'{_str} {i}'
            series = pd.Series(np.arange(self.n_genes), name=name).map(func)            
            self._gene_symbols = series
        return self._gene_symbols
    
    @property
    def is_hvg_gene(self):
        try:
            return self._is_hvg
        except AttributeError:
            is_hvg = pd.Series(np.arange(self.n_genes), name='is_hvg').map(lambda i: i % self.hvg_ratio)
            is_hvg.index = self.gene_symbols
            self._is_hvg = is_hvg
        return self._is_hvg

    @property
    def labels(self) -> pd.Series:
        try:
            return self._labels
        except AttributeError:
            _str = self.label
            name = f'{_str}s'
            func = lambda i: f'{_str} {i}'
            series = pd.Series(np.arange(self.n_points), name=name).map(func)
            self._labels = series
        return self._labels

    @property
    def conditions(self) -> pd.Series:
        try:
            return self._conditions
        except AttributeError:
            _str = self.c_str
            name = f'{_str}s'
            func = lambda i: f'{_str} {i % self.n_conditions}'
            series = pd.Series(np.arange(self.n_points), name=name).map(func)
            series.index = self.labels
            self._conditions = series
        return self._conditions
    
    @property
    def df_emb(self) -> pd.DataFrame:
        try:
            return self._df_emb
        except AttributeError:
            df_emb = pd.DataFrame(
                np.random.randn(self.n_points, self.emb_dims), 
                index=self.labels, columns=self.emb_cols
            )
            df_emb = df_emb.join(self.conditions)
            self._df_emb = df_emb
        return self._df_emb
    
    @property
    def df_cnt(self) -> pd.DataFrame:
        try:
            return self._df_cnt
        except AttributeError:
            df_cnt = pd.DataFrame(
                np.random.randn(self.n_points, self.n_genes), 
                index=self.labels, columns=self.gene_symbols
            )
            self._df_cnt = df_cnt
        return self._df_cnt
    
    @property
    def df_nrm(self) -> pd.DataFrame:
        try:
            return self._df_nrm
        except AttributeError:
            df_nrm = pd.DataFrame(
                np.random.randn(self.n_points, self.n_genes), 
                index=self.labels, columns=self.gene_symbols
            )
            self._df_nrm = df_nrm
        return self._df_nrm

    @property
    def x_emb(self):
        return f'X_{self.emb_name.lower()}'
    
    @property
    def x_norm(self):
        return f'X_{self.n_str.lower()}'

    @property
    def adata(self):
        try:
            return self._adata
        except AttributeError:
            _str = self.label
            label_name = f'{_str}s' # barcodes

            _str = self.c_str
            conds_name = f'{_str}s' # conditions

            obs = self.df_emb.reset_index()
            obs = obs[[label_name, conds_name]].set_index(label_name, drop=False)

            obsm = dict()
            obsm[self.x_emb] = self.df_emb.drop(columns=conds_name)

            layers = dict()
            layers[self.x_norm] = self.df_nrm

            adata = ad.AnnData(
                X = self.df_cnt,
                obs = obs,
                var = pd.DataFrame(self.is_hvg_gene, index=self.gene_symbols),
                obsm = obsm,
                layers = layers,
            )
            
            self._adata = adata

        return self._adata

### Processing AnnData back to DataFrames

In [ ]:
#| export


@rich_auto
@dataclass
class AnnDataProcessor:
    adata: ad.AnnData
    x_emb: str = 'X_phate'
    layer: Optional[str] = None
    

    def get_sidecars(self):
        layer = self.adata.layers.get(self.layer, None)


        if layer is None:
            layer = self.adata.X

        if hasattr(layer, 'toarray'):
            layer = layer.toarray()

        if hasattr(layer, 'todense'):
            layer = layer.todense()

        df = pd.DataFrame(
            layer,
            columns=self.adata.var.index,
            index=self.adata.obs.index
        )
        return df
    
    @property
    def emb_name(self):
        return self.x_emb.replace('X_', '').upper()

    def get_embedding(self, add_conditions:bool=False):
        emb = self.adata.obsm.get(self.x_emb, None)
        cols = [f'{self.emb_name}_{i+1}' for i in range(emb.shape[1])]
        df = pd.DataFrame(
            emb, columns=cols, 
            index=self.adata.obs.index
        )
        if add_conditions and 'conditions' in self.adata.obs.columns:        
            df.loc[:, 'conditions'] = self.adata.obs.conditions        
        return df


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()